<a href="https://colab.research.google.com/github/Dd1235/LearnAI/blob/main/mini_projects/GruSaganQuote/gru_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Dd1235/LearnAI.git
%cd LearnAI/mini_projects/GruSaganQuote

Cloning into 'LearnAI'...
remote: Enumerating objects: 34309, done.
remote: Counting objects: 100% (34309/34309), done.
remote: Compressing objects: 100% (34273/34273), done.
remote: Total 34309 (delta 53), reused 34275 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (34309/34309), 37.10 MiB | 9.96 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (32499/32499), done.
/content/LearnAI/mini_projects/GruSaganQuote


In [2]:
!python train.py

Epoch 1/30: 100% 1904/1904 [00:15<00:00, 120.84it/s]

Epoch 1/30 | Loss: 1.0120
Epoch 2/30: 100% 1904/1904 [00:15<00:00, 124.98it/s]

Epoch 2/30 | Loss: 0.6734
Epoch 3/30: 100% 1904/1904 [00:15<00:00, 121.91it/s]

Epoch 3/30 | Loss: 0.6698
Epoch 4/30: 100% 1904/1904 [00:15<00:00, 122.36it/s]

Epoch 4/30 | Loss: 0.7868
Epoch 5/30: 100% 1904/1904 [00:15<00:00, 119.98it/s]

Epoch 5/30 | Loss: 1.0554
Epoch 6/30: 100% 1904/1904 [00:16<00:00, 118.59it/s]

Epoch 6/30 | Loss: 1.0971
Epoch 7/30: 100% 1904/1904 [00:15<00:00, 119.65it/s]

Epoch 7/30 | Loss: 1.0335
Epoch 8/30: 100% 1904/1904 [00:15<00:00, 120.43it/s]

Epoch 8/30 | Loss: 1.0532
Epoch 9/30: 100% 1904/1904 [00:15<00:00, 121.15it/s]

Epoch 9/30 | Loss: 1.0399
Epoch 10/30: 100% 1904/1904 [00:15<00:00, 120.56it/s]

Epoch 10/30 | Loss: 1.1302
Epoch 11/30: 100% 1904/1904 [00:15<00:00, 119.96it/s]

Epoch 11/30 | Loss: 1.3405
Epoch 12/30: 100% 1904/1904 [00:15<00:00, 120.73it/s]

Epoch 12/30 | Loss: 2.0510
Epoch 13/30: 100% 1904/1904 [00:15

In [3]:
from google.colab import files
files.download("sagan_gru.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
import torch
import torch.nn as nn
from model.gru_model import CharGRU
from utils import *

In [5]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Running on device: {device}")

Running on device: cuda


In [6]:
text = load_and_clean_text("carl_sagan_quotes.txt")
char2idx, idx2char = create_char_mappings(text)

In [7]:
hidden_size = 256
model = CharGRU(len(char2idx), hidden_size)
model.load_state_dict(torch.load("sagan_gru.pth", map_location=device))
model.to(device)
model.eval()

CharGRU(
  (embed): Embedding(47, 256)
  (gru): GRU(256, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=47, bias=True)
)

In [8]:
def generate(model, start, char2idx, idx2char, length=300, temperature=1.0, device="cpu"):
    model.eval()
    input_seq = torch.tensor([char2idx[c] for c in start.lower()], dtype=torch.long).unsqueeze(0).to(device)
    hidden = model.init_hidden(1).to(device)
    generated = start

    for _ in range(length):
        output, hidden = model(input_seq, hidden)
        output_dist = torch.softmax(output[-1] / temperature, dim=0)
        char_id = torch.multinomial(output_dist, 1).item()
        next_char = idx2char[char_id]
        generated += next_char
        input_seq = torch.tensor([[char_id]], dtype=torch.long).to(device)

    return generated

In [10]:
seed = "we are"
output = generate(model, seed, char2idx, idx2char, length=300, temperature=0.8, device=device)
print(f"\nGenerated Text:\n{output}")


Generated Text:
we are on the entire umitical. that's insights in the plong to believe so a places if we moment to me, in the the futur did byou lon the enduracialting evidence. school decly usid religion. in reatend more gaspenderequire vistant indistinguish, there athat sound that resusem to the earth numos...howring t


In [11]:
generate(model, "on a moat", char2idx, idx2char, length=300, temperature=0.8, device=device)

"on a moater, well, so mindolate bill are bigor public qued hore a devers that many of the intelligence tend to support attempt tuals the fores. we've for sufficienting. our persones, the same sense for great arsuross the one has or ever god served have a planet for ones to find it hage quent their has ever m"

In [16]:
output = generate(model, seed, char2idx, idx2char, length=300, temperature=0.4, device=device)

In [17]:
output # lower temperature gives more coherant sentances

'we are the are the stars of science and the extray, but in many the fincledition is to be so than the imple the stars in a nation. in the end more sometimes reepletes and beginnal belief; if there on a place for the universe is made of the earth of mehow vast its and that it was wonder sometimes reason an'

In [18]:
generate(model, seed, char2idx, idx2char, length=300, temperature=0.1, device=device)

'we are the could be so to thing our contrary to the inside of the stars. that are all of our progress have the cosmos are not in the universe is a predisch in a so in is a sunce of the see that the seems to me of the cosmos arguments and the same to be crazy, of new hypotheses have been universe in the un'

In [19]:
generate(model, "on a moat", char2idx, idx2char, length=300, temperature=0.2, device=device)

'on a moater the courage to the strul. for the idea of science is an are are are more construct ideas of the stars. there is a say for such a god it is a system technology that the standard its understo of our planet in a so get to be crite to set of the see that the stars. there is a religions in the in the '